In [1]:
from keras.layers import LSTM, Dense, Conv1D, Dropout, GlobalMaxPool1D
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import pickle

Using plaidml.keras.backend backend.


In [2]:
model_type = 'EDM'
def get_pickles(pick_name): 
    x_train, x_test, y_train, y_test, notes_dict = pickle.load(open(f'../Pickles/{pick_name}', 'rb'))
    
    return x_train, x_test, y_train, y_test, notes_dict


x_train, x_test, y_train, y_test, notes_dict = get_pickles(f'{model_type}.p')
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(2004, 10, 1) (2004,)
(175, 10, 1) (175,)


In [3]:
# x_train = x_train[:5001]
# y_train = y_train[:5001]
# x_test = x_test[:5001]
# y_test = y_test[:5001]
# print(x_train.shape, y_train.shape)
# print(x_test.shape, y_test.shape)

In [4]:
def get_lstm(n): 
    model = Sequential()
    model.add(LSTM(256, return_sequences = True))
    model.add(LSTM(256))
    model.add(Dropout(.25))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(.25))
    model.add(Dense(n, activation = 'softmax')) 
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model


def get_lstm(n): 
    model = Sequential()
    model.add(LSTM(128, return_sequences = True))
    model.add(LSTM(128))    
#     model.add(Dense(128, activation = 'relu'))
#     model.add(Dropout(.3))
    model.add(Dense(256, activation = 'relu'))
#     model.add(Dense(512, activation = 'relu'))

    model.add(Dropout(.3))
    model.add(Dense(n, activation = 'softmax')) 
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [5]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint(f'ModelWeights/LSTM_{model_type}.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')

batch = 128 
epochs = 1000 
callbacks = [early_stopping, model_checkpoint]

unique_n = len(notes_dict)
lstm_model = get_lstm(unique_n)

lstm_history = lstm_model.fit(x_train, y_train, batch_size = batch, epochs = epochs, validation_data = (x_test, y_test), 
                        callbacks = callbacks)

pickle.dump(lstm_history, open(f'ModelPerf/{model_type}.p', 'wb'))

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


Train on 2004 samples, validate on 175 samples
Epoch 1/1000
2004/2004 [==============================] - 24s 12ms/step - loss: 5.5842 - acc: 0.0070 - val_loss: 5.2724 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 5.27236, saving model to ModelWeights/LSTM_EDM.h5
Epoch 2/1000
2004/2004 [==============================] - 0s 214us/step - loss: 5.1984 - acc: 0.0145 - val_loss: 5.1869 - val_acc: 0.0400

Epoch 00002: val_loss improved from 5.27236 to 5.18690, saving model to ModelWeights/LSTM_EDM.h5
Epoch 3/1000
2004/2004 [==============================] - 0s 209us/step - loss: 5.1044 - acc: 0.0135 - val_loss: 5.1709 - val_acc: 0.0000e+00

Epoch 00003: val_loss improved from 5.18690 to 5.17093, saving model to ModelWeights/LSTM_EDM.h5
Epoch 4/1000
2004/2004 [==============================] - 0s 214us/step - loss: 5.0545 - acc: 0.0170 - val_loss: 5.1365 - val_acc: 0.0000e+00

Epoch 00004: val_loss improved from 5.17093 to 5.13647, saving model to ModelWeights/LSTM_EDM.h5
E

In [6]:
assert False

AssertionError: 

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:

import os
os.system(r'%windir%\system32\rundll32.exe powrprof.dll,SetSuspendState Hibernate')